## 🔄 Data Refresh & Setup

Run this cell to fetch the latest data and initialize the dashboard.

In [1]:
# Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add src to path - handle running from notebooks/ directory
if Path.cwd().name == 'notebooks':
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()
sys.path.insert(0, str(project_root / 'src'))

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Import custom modules
from data_collection.fred_data_fetcher import FREDDataFetcher
from processing.recession_analyzer import RecessionIndicatorAnalyzer
from processing.recession_markers import RecessionMarkers

print("📦 Libraries loaded successfully!")
print(f"⏰ Dashboard initialized at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


📦 Libraries loaded successfully!
⏰ Dashboard initialized at: 2026-01-08 17:44:41


In [2]:
# Fetch latest data
print("🔄 Fetching latest economic data...\n")

fetcher = FREDDataFetcher()
data = fetcher.fetch_all_indicators(start_date='2000-01-01')

print("\n✅ Data fetch complete!")
print(f"📊 Total indicators: {sum(len(df.columns) for df in data.values())}")
print(f"📅 Date range: {min(df.index.min() for df in data.values()).strftime('%Y-%m-%d')} to {max(df.index.max() for df in data.values()).strftime('%Y-%m-%d')}")

🔄 Fetching latest economic data...


FETCHING ALL RECESSION INDICATORS (CORE + SECONDARY)

FETCHING ALL CORE RECESSION INDICATORS

=== Fetching Treasury Yields ===
Loading DGS10 from cache...
Loading DGS2 from cache...
Loading DGS3MO from cache...

=== Fetching Labor Market Data ===
Loading UNRATE from cache...
Loading SAHMREALTIME from cache...
Loading ICSA from cache...

=== Fetching Credit Spreads ===
Loading BAMLH0A0HYM2 from cache...
Loading BAMLC0A4CBBB from cache...
Loading DBAA from cache...
Loading DAAA from cache...
Loading DGS10 from cache...

DATA FETCH COMPLETE

FETCHING ALL SECONDARY RECESSION INDICATORS

=== Fetching Leading Economic Index ===
Loading USSLIND from cache...

=== Fetching Manufacturing PMI ===
Loading MANEMP from cache...
Loading NEWORDER from cache...
Fetching NAPM from FRED API...
Error fetching NAPM: Bad Request.  The series does not exist.
NAPM not available
Loading INDPRO from cache...

=== Fetching GDP Data ===
Loading GDPC1 from cache...
Loading A93

## 🎯 Current Recession Risk Assessment

Composite risk score based on multiple economic indicators.

In [3]:
# Analyze current recession risk
analyzer = RecessionIndicatorAnalyzer()
analysis = analyzer.analyze_all_indicators(data)

composite = analysis['composite']
score = composite['composite_score']

# Display risk gauge
fig = go.Figure(go.Indicator(
    mode = "gauge+number+delta",
    value = score,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Recession Risk Score", 'font': {'size': 24}},
    delta = {'reference': 50},
    gauge = {
        'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': "darkgray"},
        'bar': {'color': composite['risk_color']},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 25], 'color': 'rgba(0, 255, 0, 0.2)'},
            {'range': [25, 50], 'color': 'rgba(255, 255, 0, 0.2)'},
            {'range': [50, 75], 'color': 'rgba(255, 165, 0, 0.2)'},
            {'range': [75, 100], 'color': 'rgba(255, 0, 0, 0.2)'}],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': 75}
    }
))

fig.update_layout(
    height=400,
    font={'size': 16}
)

fig.show()

# Display risk level
print(f"\n{'='*70}")
print(f"CURRENT RECESSION RISK: {score:.1f}/100")
print(f"STATUS: {composite['risk_level']}")
print(f"{'='*70}")


CURRENT RECESSION RISK: 53.7/100
STATUS: Warning - Elevated Risk


## 📋 Indicator Breakdown

Individual scores for each recession indicator.

In [4]:
# Create indicator breakdown table
breakdown_data = []
for indicator, details in sorted(composite['breakdown'].items(), 
                                 key=lambda x: x[1]['contribution'], 
                                 reverse=True):
    breakdown_data.append({
        'Indicator': details['description'],
        'Score': f"{details['score']:.1f}/100",
        'Weight': f"{details['weight']}%",
        'Contribution': f"{details['contribution']:.1f}",
        'Signal': details['signal']
    })

breakdown_df = pd.DataFrame(breakdown_data)

# Display as table
print("\n📊 INDICATOR BREAKDOWN:\n")
print(breakdown_df.to_string(index=False))

# Create bar chart of contributions
fig = go.Figure()

fig.add_trace(go.Bar(
    x=[d['Contribution'] for d in breakdown_data],
    y=[d['Indicator'] for d in breakdown_data],
    orientation='h',
    marker=dict(
        color=[composite['breakdown'][ind]['score'] for ind in composite['breakdown'].keys()],
        colorscale='RdYlGn_r',
        showscale=True,
        colorbar=dict(title="Score")
    ),
    text=[d['Signal'] for d in breakdown_data],
    textposition='auto'
))

fig.update_layout(
    title='Indicator Contributions to Composite Score',
    xaxis_title='Contribution to Total Risk',
    yaxis_title='',
    height=400,
    showlegend=False
)

fig.show()


📊 INDICATOR BREAKDOWN:

                    Indicator    Score Weight Contribution                          Signal
          Sahm Rule Indicator 82.5/100    25%         20.6 Warning - Approaching Threshold
Unemployment Rate Change (3m) 75.0/100    20%         15.0                Warning - Rising
       10Y-2Y Treasury Spread 27.9/100    25%          7.0                          Normal
             GDP Growth (QoQ) 39.3/100    15%          5.9                          Normal
     High Yield Credit Spread 34.9/100    15%          5.2                          Normal


## 📈 Core Indicators: Treasury Yields & Yield Curve

The yield curve is one of the most reliable recession predictors.

In [5]:
# Treasury yields and spread
ty = data['treasury_yields']
markers = RecessionMarkers()

# Create subplots
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Treasury Yields', '10Y-2Y Spread (Yield Curve)'),
    vertical_spacing=0.12,
    row_heights=[0.5, 0.5]
)

# Plot yields
fig.add_trace(
    go.Scatter(x=ty.index, y=ty['DGS10'], mode='lines', name='10-Year',
               line=dict(color='blue', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=ty.index, y=ty['DGS2'], mode='lines', name='2-Year',
               line=dict(color='green', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=ty.index, y=ty['DGS3MO'], mode='lines', name='3-Month',
               line=dict(color='orange', width=2)),
    row=1, col=1
)

# Plot spread
fig.add_trace(
    go.Scatter(x=ty.index, y=ty['Spread_10Y2Y'], mode='lines', name='10Y-2Y Spread',
               line=dict(color='purple', width=3)),
    row=2, col=1
)

# Add zero line
fig.add_hline(y=0, line_dash="dash", line_color="red", row=2, col=1,
              annotation_text="Inversion Threshold", annotation_position="right")

# Add recession shading
recessions = markers.get_recession_periods(ty.index.min(), ty.index.max())
for _, rec in recessions.iterrows():
    # Top panel
    fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                  layer="below", line_width=0, row=1, col=1)
    # Bottom panel  
    fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                  layer="below", line_width=0, row=2, col=1)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Yield (%)", row=1, col=1)
fig.update_yaxes(title_text="Spread (%)", row=2, col=1)

fig.update_layout(height=700, showlegend=True, hovermode='x unified')
fig.show()

# Current values
current_spread = ty['Spread_10Y2Y'].dropna().iloc[-1]
spread_date = ty['Spread_10Y2Y'].dropna().index[-1]
print(f"\n📊 Current 10Y-2Y Spread: {current_spread:+.2f}% (as of {spread_date.strftime('%Y-%m-%d')})")
if current_spread < 0:
    print("⚠️  INVERTED - Historical recession signal!")
else:
    print("✅ Normal curve")


📊 Current 10Y-2Y Spread: +0.71% (as of 2026-01-06)
✅ Normal curve


## 👥 Labor Market Indicators

Unemployment and the Sahm Rule recession indicator.

In [6]:
# Labor market data
lm = data['labor_market']

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Unemployment Rate', 'Sahm Rule Recession Indicator'),
    vertical_spacing=0.12
)

# Unemployment rate
fig.add_trace(
    go.Scatter(x=lm.index, y=lm['UNRATE'], mode='lines', name='Unemployment Rate',
               line=dict(color='blue', width=2)),
    row=1, col=1
)

# Sahm Rule
fig.add_trace(
    go.Scatter(x=lm.index, y=lm['SAHM_Rule'], mode='lines', name='Sahm Rule',
               line=dict(color='red', width=3)),
    row=2, col=1
)

# Add recession threshold
fig.add_hline(y=0.5, line_dash="dash", line_color="red", row=2, col=1,
              annotation_text="Recession Threshold (0.5)", annotation_position="right")

# Add recession shading
for _, rec in recessions.iterrows():
    fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                  layer="below", line_width=0, row=1, col=1)
    fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                  layer="below", line_width=0, row=2, col=1)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Unemployment Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Sahm Rule Value", row=2, col=1)

fig.update_layout(height=700, showlegend=True, hovermode='x unified')
fig.show()

# Current values
current_unrate = lm['UNRATE'].dropna().iloc[-1]
current_sahm = lm['SAHM_Rule'].dropna().iloc[-1]
sahm_date = lm['SAHM_Rule'].dropna().index[-1]

print(f"\n📊 Current Unemployment: {current_unrate:.1f}%")
print(f"📊 Current Sahm Rule: {current_sahm:.2f} (as of {sahm_date.strftime('%Y-%m-%d')})")
if current_sahm >= 0.5:
    print("🚨 RECESSION SIGNAL TRIGGERED!")
elif current_sahm >= 0.3:
    print("⚠️  Approaching recession threshold")
else:
    print("✅ Below recession threshold")


📊 Current Unemployment: 4.6%
📊 Current Sahm Rule: 0.43 (as of 2025-11-01)
⚠️  Approaching recession threshold


## 💰 Credit Spreads

Corporate bond spreads indicate financial stress.

In [7]:
# Credit spreads
cs = data['credit_spreads']

fig = go.Figure()

# Plot available spreads
if 'HY_Spread' in cs.columns:
    fig.add_trace(go.Scatter(x=cs.index, y=cs['HY_Spread'], mode='lines',
                             name='High Yield Spread', line=dict(width=2)))

if 'BAA_Spread' in cs.columns:
    fig.add_trace(go.Scatter(x=cs.index, y=cs['BAA_Spread'], mode='lines',
                             name='BAA Spread', line=dict(width=2)))

if 'BBB_Spread' in cs.columns:
    fig.add_trace(go.Scatter(x=cs.index, y=cs['BBB_Spread'], mode='lines',
                             name='BBB Spread', line=dict(width=2)))

# Add stress thresholds
fig.add_hline(y=4.0, line_dash="dash", line_color="orange",
              annotation_text="Elevated Stress (4%)", annotation_position="right")
fig.add_hline(y=6.0, line_dash="dash", line_color="red",
              annotation_text="Severe Stress (6%)", annotation_position="right")

# Add recession shading
for _, rec in recessions.iterrows():
    fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                  layer="below", line_width=0)

fig.update_layout(
    title='Corporate Credit Spreads',
    xaxis_title='Date',
    yaxis_title='Spread (%)',
    height=500,
    hovermode='x unified'
)

fig.show()

# Current values
print("\n📊 Current Credit Spreads:")
for col in ['HY_Spread', 'BAA_Spread', 'BBB_Spread']:
    if col in cs.columns:
        current = cs[col].dropna().iloc[-1]
        date = cs[col].dropna().index[-1]
        print(f"  {col}: {current:.2f}% (as of {date.strftime('%Y-%m-%d')})")


📊 Current Credit Spreads:
  HY_Spread: 2.79% (as of 2026-01-07)
  BAA_Spread: 1.74% (as of 2026-01-06)
  BBB_Spread: 1.01% (as of 2026-01-07)


## 📊 Secondary Indicators Overview

In [8]:
# GDP Growth
gdp = data['gdp']

fig = go.Figure()

fig.add_trace(go.Scatter(x=gdp.index, y=gdp['GDP_YoY_Growth'], mode='lines',
                         name='GDP YoY Growth', line=dict(color='green', width=2)))

fig.add_hline(y=0, line_dash="dash", line_color="red",
              annotation_text="Zero Growth", annotation_position="right")

# Add recession shading
for _, rec in recessions.iterrows():
    fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                  layer="below", line_width=0)

fig.update_layout(
    title='Real GDP Growth (Year-over-Year)',
    xaxis_title='Date',
    yaxis_title='Growth Rate (%)',
    height=400,
    hovermode='x unified'
)

fig.show()

current_gdp = gdp['GDP_YoY_Growth'].dropna().iloc[-1]
gdp_date = gdp['GDP_YoY_Growth'].dropna().index[-1]
print(f"\n📊 Current GDP Growth: {current_gdp:+.2f}% YoY (as of {gdp_date.strftime('%Y-%m-%d')})")


📊 Current GDP Growth: +2.33% YoY (as of 2025-07-01)


In [9]:
# Consumer Confidence
consumer = data['consumer']

fig = go.Figure()

fig.add_trace(go.Scatter(x=consumer.index, y=consumer['UMich_Sentiment'], mode='lines',
                         name='U. Michigan Sentiment', line=dict(color='blue', width=2)))

# Add recession shading
for _, rec in recessions.iterrows():
    fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                  layer="below", line_width=0)

fig.update_layout(
    title='Consumer Sentiment Index',
    xaxis_title='Date',
    yaxis_title='Index Value',
    height=400,
    hovermode='x unified'
)

fig.show()

current_sentiment = consumer['UMich_Sentiment'].dropna().iloc[-1]
sentiment_date = consumer['UMich_Sentiment'].dropna().index[-1]
print(f"\n📊 Current Consumer Sentiment: {current_sentiment:.1f} (as of {sentiment_date.strftime('%Y-%m-%d')})")


📊 Current Consumer Sentiment: 51.0 (as of 2025-11-01)


In [10]:
# Enhanced risk scorecard with all details
scorecard_data = []

for indicator_key in composite['breakdown'].keys():
    details = composite['breakdown'][indicator_key]
    
    scorecard_data.append({
        'Indicator': details['description'],
        'Current Signal': details['signal'],
        'Risk Score': f"{details['score']:.1f}/100",
        'Weight': f"{details['weight']}%",
        'Weighted Contribution': f"{details['contribution']:.1f}",
        'Risk Level': (
            '🟢 Normal' if details['score'] < 25 else
            '🟡 Caution' if details['score'] < 50 else
            '🟠 Warning' if details['score'] < 75 else
            '🔴 Critical'
        )
    })

scorecard_df = pd.DataFrame(scorecard_data)
scorecard_df = scorecard_df.sort_values('Weighted Contribution', ascending=False)

print("\n" + "="*120)
print("ENHANCED RECESSION RISK SCORECARD")
print("="*120)
print(scorecard_df.to_string(index=False))
print("="*120)
print(f"\nCOMPOSITE SCORE: {composite['composite_score']:.1f}/100")
print(f"RISK LEVEL: {composite['risk_level']}")
print("="*120)

# Create heatmap-style visualization
fig = go.Figure(data=go.Heatmap(
    z=[scorecard_df['Risk Score'].str.replace('/100', '').astype(float).tolist()],
    x=scorecard_df['Indicator'].tolist(),
    y=['Risk Score'],
    colorscale='RdYlGn_r',
    text=[scorecard_df['Current Signal'].tolist()],
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Score (0-100)")
))

fig.update_layout(
    title='Risk Heatmap - All Indicators',
    xaxis_title='',
    yaxis_title='',
    height=300,
    xaxis_tickangle=-45
)

fig.show()


ENHANCED RECESSION RISK SCORECARD
                    Indicator                  Current Signal Risk Score Weight Weighted Contribution Risk Level
       10Y-2Y Treasury Spread                          Normal   27.9/100    25%                   7.0  🟡 Caution
             GDP Growth (QoQ)                          Normal   39.3/100    15%                   5.9  🟡 Caution
     High Yield Credit Spread                          Normal   34.9/100    15%                   5.2  🟡 Caution
          Sahm Rule Indicator Warning - Approaching Threshold   82.5/100    25%                  20.6 🔴 Critical
Unemployment Rate Change (3m)                Warning - Rising   75.0/100    20%                  15.0  🟠 Warning

COMPOSITE SCORE: 53.7/100
RISK LEVEL: Warning - Elevated Risk


## 🎯 Enhanced Risk Scorecard

Detailed breakdown showing all indicators with weights and current status.

In [11]:
# Trend analysis: Calculate recent trends and volatility
from scipy import stats

trend_data = []

# Analyze key indicators
trend_indicators = [
    ('treasury_yields', 'Spread_10Y2Y', '10Y-2Y Spread'),
    ('labor_market', 'UNRATE', 'Unemployment Rate'),
    ('credit_spreads', 'BAA_Spread', 'BAA Credit Spread'),
    ('gdp', 'GDP_YoY_Growth', 'GDP Growth'),
    ('consumer', 'UMich_Sentiment', 'Consumer Sentiment'),
    ('lei', 'LEI_6M_Change', 'LEI 6M Change'),
    ('pmi', 'ISM_PMI', 'ISM PMI'),
]

for category, indicator, name in trend_indicators:
    if category not in data or indicator not in data[category].columns:
        continue
    
    series = data[category][indicator].dropna()
    
    # Skip if no data
    if len(series) == 0:
        continue
    
    # Get last 12 months of data
    cutoff_date = series.index[-1] - pd.DateOffset(months=12)
    recent_data = series[series.index >= cutoff_date]
    
    if len(recent_data) < 2:
        continue
    
    # Calculate trend (linear regression slope)
    x = np.arange(len(recent_data))
    y = recent_data.values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    
    # Annualized trend
    periods_per_year = 12 if len(recent_data) > 20 else 4  # Monthly or quarterly
    annualized_trend = slope * periods_per_year
    
    # Calculate volatility (standard deviation)
    volatility = recent_data.std()
    
    # Current value
    current = recent_data.iloc[-1]
    
    # 12-month change
    change_12m = current - recent_data.iloc[0]
    
    # Trend direction
    if abs(annualized_trend) < volatility * 0.1:
        direction = '➡️ Flat'
    elif annualized_trend > 0:
        direction = '📈 Rising'
    else:
        direction = '📉 Falling'
    
    trend_data.append({
        'Indicator': name,
        'Current': f"{current:.2f}",
        '12M Change': f"{change_12m:+.2f}",
        'Trend': direction,
        'Annualized Trend': f"{annualized_trend:+.2f}",
        'Volatility (σ)': f"{volatility:.2f}",
        'R²': f"{r_value**2:.3f}"
    })

trend_df = pd.DataFrame(trend_data)

print("\n" + "="*120)
print("TREND ANALYSIS - LAST 12 MONTHS")
print("="*120)
print(trend_df.to_string(index=False))
print("="*120)
print("\nInterpretation:")
print("  📈 Rising: Indicator trending upward")
print("  📉 Falling: Indicator trending downward")
print("  ➡️  Flat: No significant trend")
print("  R²: Strength of trend (1.0 = perfect linear trend)")
print("  Volatility (σ): Standard deviation - higher values indicate more variability")
print("="*120)


TREND ANALYSIS - LAST 12 MONTHS
         Indicator Current 12M Change     Trend Annualized Trend Volatility (σ)    R²
     10Y-2Y Spread    0.71      +0.37  📈 Rising            +0.02           0.12 0.645
 Unemployment Rate    4.60      +0.40  📈 Rising            +0.13           0.16 0.558
 BAA Credit Spread    1.74      +0.30   ➡️ Flat            +0.01           0.13 0.151
        GDP Growth    2.33      +0.31  📈 Rising            +0.62           0.16 0.892
Consumer Sentiment   51.00     -20.80 📉 Falling            -6.48           8.04 0.615

Interpretation:
  📈 Rising: Indicator trending upward
  📉 Falling: Indicator trending downward
  ➡️  Flat: No significant trend
  R²: Strength of trend (1.0 = perfect linear trend)
  Volatility (σ): Standard deviation - higher values indicate more variability


## 📈 Trend Analysis & Summary Statistics

Statistical analysis of recent trends and volatility.

In [12]:
# Calculate pre-recession averages and compare to current values
# Get NBER recessions from markers object
recession_periods = markers.get_recession_periods(pd.Timestamp('1960-01-01'), pd.Timestamp('2025-12-31'))

# Define key indicators to compare
comparison_indicators = [
    ('treasury_yields', 'Spread_10Y2Y', '10Y-2Y Spread', '%'),
    ('labor_market', 'UNRATE', 'Unemployment Rate', '%'),
    ('labor_market', 'SAHM_Rule', 'Sahm Rule', ''),
    ('credit_spreads', 'BAA_Spread', 'BAA Spread', '%'),
    ('gdp', 'GDP_YoY_Growth', 'GDP Growth (YoY)', '%'),
    ('consumer', 'UMich_Sentiment', 'Consumer Sentiment', ''),
    ('lei', 'LEI_6M_Change', 'LEI 6M Change', '%'),
    ('pmi', 'ISM_PMI', 'ISM PMI', ''),
]

comparison_data = []

for category, indicator, name, unit in comparison_indicators:
    if category not in data or indicator not in data[category].columns:
        continue
    
    series = data[category][indicator].dropna()
    
    if len(series) == 0:
        continue
    
    # Current value
    current_value = series.iloc[-1]
    current_date = series.index[-1]
    
    # Calculate pre-recession averages (6 months before each recession)
    pre_recession_values = []
    
    for _, rec in recession_periods.iterrows():
        rec_start = pd.Timestamp(rec['start'])
        pre_start = rec_start - pd.DateOffset(months=6)
        pre_end = rec_start
        
        # Get data for 6 months before recession
        pre_rec_data = series[(series.index >= pre_start) & (series.index < pre_end)]
        if len(pre_rec_data) > 0:
            pre_recession_values.append(pre_rec_data.mean())
    
    if len(pre_recession_values) > 0:
        avg_pre_recession = np.mean(pre_recession_values)
        
        # Calculate difference
        diff = current_value - avg_pre_recession
        pct_diff = (diff / abs(avg_pre_recession) * 100) if avg_pre_recession != 0 else 0
        
        comparison_data.append({
            'Indicator': name,
            'Current': f"{current_value:.2f}{unit}",
            'Pre-Recession Avg': f"{avg_pre_recession:.2f}{unit}",
            'Difference': f"{diff:+.2f}{unit}",
            'Status': '🟢 Better' if (
                ('Spread' in name and diff > 0) or
                ('Unemployment' in name and diff < 0) or
                ('Sahm' in name and diff < 0) or
                ('Credit' in name and diff < 0) or
                ('Growth' in name and diff > 0) or
                ('Sentiment' in name and diff > 0) or
                ('LEI' in name and diff > 0) or
                ('PMI' in name and diff > 0)
            ) else '🔴 Worse'
        })

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*100)
print("CURRENT VALUES vs PRE-RECESSION AVERAGES")
print("="*100)
print("Pre-recession average calculated from 6 months before each NBER recession start date")
print("="*100)
print(comparison_df.to_string(index=False))
print("="*100)

# Create visualization
fig = go.Figure()

indicators_list = comparison_df['Indicator'].tolist()
current_vals = [float(str(v).replace('%', '').replace('+', '')) for v in comparison_df['Current']]
pre_rec_vals = [float(str(v).replace('%', '').replace('+', '')) for v in comparison_df['Pre-Recession Avg']]

x = np.arange(len(indicators_list))
width = 0.35

fig.add_trace(go.Bar(
    name='Current',
    x=indicators_list,
    y=current_vals,
    marker_color='blue'
))

fig.add_trace(go.Bar(
    name='Pre-Recession Avg',
    x=indicators_list,
    y=pre_rec_vals,
    marker_color='red'
))

fig.update_layout(
    title='Current Indicators vs Pre-Recession Historical Averages',
    xaxis_title='Indicator',
    yaxis_title='Value',
    barmode='group',
    height=500,
    xaxis_tickangle=-45
)

fig.show()


CURRENT VALUES vs PRE-RECESSION AVERAGES
Pre-recession average calculated from 6 months before each NBER recession start date
         Indicator Current Pre-Recession Avg Difference  Status
         Sahm Rule    0.43              0.00      +0.43 🔴 Worse
Consumer Sentiment   51.00             99.80     -48.80 🔴 Worse


## 📊 Historical Comparison: Current vs Pre-Recession Averages

Compare current indicator values to their averages in the 6 months before each recession.

In [13]:
# Housing indicators
housing = data['housing']

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Housing Starts', 'Building Permits', 'New Home Sales', 'Existing Home Sales'),
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

# Housing starts
fig.add_trace(
    go.Scatter(x=housing.index, y=housing['Housing_Starts'], mode='lines',
               name='Starts', line=dict(color='blue', width=2)),
    row=1, col=1
)

# Building permits
fig.add_trace(
    go.Scatter(x=housing.index, y=housing['Building_Permits'], mode='lines',
               name='Permits', line=dict(color='green', width=2)),
    row=1, col=2
)

# New home sales
if 'New_Home_Sales' in housing.columns:
    fig.add_trace(
        go.Scatter(x=housing.index, y=housing['New_Home_Sales'], mode='lines',
                   name='New Sales', line=dict(color='orange', width=2)),
        row=2, col=1
    )

# Existing home sales
if 'Existing_Home_Sales' in housing.columns:
    fig.add_trace(
        go.Scatter(x=housing.index, y=housing['Existing_Home_Sales'], mode='lines',
                   name='Existing Sales', line=dict(color='purple', width=2)),
        row=2, col=2
    )

# Add recession shading to all subplots
for _, rec in recessions.iterrows():
    for row in [1, 2]:
        for col in [1, 2]:
            fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                          layer="below", line_width=0, row=row, col=col)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_yaxes(title_text="Thousands", row=1, col=1)
fig.update_yaxes(title_text="Thousands", row=1, col=2)
fig.update_yaxes(title_text="Thousands", row=2, col=1)
fig.update_yaxes(title_text="Thousands", row=2, col=2)

fig.update_layout(height=700, showlegend=False, hovermode='x unified')
fig.show()

# Current values
print("\n📊 Current Housing Indicators:")
for col in ['Housing_Starts', 'Building_Permits', 'New_Home_Sales', 'Existing_Home_Sales']:
    if col in housing.columns:
        current = housing[col].dropna().iloc[-1]
        date = housing[col].dropna().index[-1]
        print(f"  {col.replace('_', ' ')}: {current:.0f}K (as of {date.strftime('%Y-%m-%d')})")


📊 Current Housing Indicators:
  Housing Starts: 1307K (as of 2025-08-01)
  Building Permits: 1330K (as of 2025-08-01)
  New Home Sales: 800K (as of 2025-08-01)
  Existing Home Sales: 4130000K (as of 2025-11-01)


## 🏠 Housing Market Indicators

Housing starts, permits, and sales are leading indicators of economic activity.

In [14]:
# Manufacturing PMI
if 'pmi' in data and len(data['pmi']) > 0:
    pmi = data['pmi']
    
    fig = go.Figure()
    
    if 'ISM_PMI' in pmi.columns:
        fig.add_trace(go.Scatter(x=pmi.index, y=pmi['ISM_PMI'], mode='lines',
                                 name='ISM Manufacturing PMI', line=dict(color='brown', width=2)))
    
        # Add expansion/contraction threshold
        fig.add_hline(y=50, line_dash="dash", line_color="black",
                      annotation_text="Expansion/Contraction Threshold (50)", annotation_position="right")
    
        # Add recession shading
        for _, rec in recessions.iterrows():
            fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                          layer="below", line_width=0)
    
        fig.update_layout(
            title='ISM Manufacturing PMI',
            xaxis_title='Date',
            yaxis_title='PMI Value',
            height=400,
            hovermode='x unified'
        )
    
        fig.show()
    
        current_pmi = pmi['ISM_PMI'].dropna().iloc[-1]
        pmi_date = pmi['ISM_PMI'].dropna().index[-1]
    
        print(f"\n📊 Current PMI: {current_pmi:.1f} (as of {pmi_date.strftime('%Y-%m-%d')})")
        if current_pmi < 50:
            print("🚨 CONTRACTION - Manufacturing sector shrinking")
        elif current_pmi < 52:
            print("⚠️  Marginal expansion")
        else:
            print("✅ Expanding")
    else:
        print("\n⚠️  ISM PMI data not available")
else:
    print("\n⚠️  PMI data not available in dataset")


⚠️  PMI data not available in dataset


## 🏭 Manufacturing PMI

The ISM Manufacturing PMI is a key gauge of manufacturing sector health.

In [15]:
# Leading Economic Index
if 'lei' in data and len(data['lei']) > 0:
    lei = data['lei']
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Leading Economic Index', 'LEI 6-Month Change'),
        vertical_spacing=0.12
    )
    
    # LEI level
    if 'LEI' in lei.columns:
        fig.add_trace(
            go.Scatter(x=lei.index, y=lei['LEI'], mode='lines', name='LEI',
                       line=dict(color='purple', width=2)),
            row=1, col=1
        )
    
    # LEI 6-month change
    if 'LEI_6M_Change' in lei.columns:
        fig.add_trace(
            go.Scatter(x=lei.index, y=lei['LEI_6M_Change'], mode='lines', name='6M Change',
                       line=dict(color='red', width=2)),
            row=2, col=1
        )
    
        # Add threshold for 6M change
        fig.add_hline(y=-2.0, line_dash="dash", line_color="red", row=2, col=1,
                      annotation_text="Recession Signal (-2%)", annotation_position="right")
    
    # Add recession shading
    for _, rec in recessions.iterrows():
        fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                      layer="below", line_width=0, row=1, col=1)
        fig.add_vrect(x0=rec['start'], x1=rec['end'], fillcolor="red", opacity=0.1,
                      layer="below", line_width=0, row=2, col=1)
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Index Value", row=1, col=1)
    fig.update_yaxes(title_text="6M % Change", row=2, col=1)
    
    fig.update_layout(height=700, showlegend=True, hovermode='x unified')
    fig.show()
    
    # Check if we have data before accessing values
    lei_data = lei['LEI'].dropna() if 'LEI' in lei.columns else pd.Series()
    lei_6m_data = lei['LEI_6M_Change'].dropna() if 'LEI_6M_Change' in lei.columns else pd.Series()
    
    if len(lei_data) > 0 and len(lei_6m_data) > 0:
        current_lei = lei_data.iloc[-1]
        current_lei_6m = lei_6m_data.iloc[-1]
        lei_date = lei_data.index[-1]
    
        print(f"\n📊 Current LEI: {current_lei:.1f} (as of {lei_date.strftime('%Y-%m-%d')})")
        print(f"📊 6-Month Change: {current_lei_6m:+.2f}%")
        if current_lei_6m < -2.0:
            print("🚨 RECESSION SIGNAL - LEI declining rapidly!")
        elif current_lei_6m < 0:
            print("⚠️  Declining trend")
        else:
            print("✅ Positive growth")
    else:
        print("\n⚠️  LEI indicators not fully available (no data after filtering)")
else:
    print("\n⚠️  LEI data not available in dataset")


⚠️  LEI indicators not fully available (no data after filtering)


## 📊 Leading Economic Index (LEI)

The Conference Board's LEI is a composite indicator designed to predict economic turning points.

## 📅 Latest Values Summary Table

In [16]:
# Create summary table of latest values
latest = fetcher.get_latest_values(include_secondary=True)

summary_data = []

# Key indicators
key_indicators = [
    ('treasury_yields', 'Spread_10Y2Y', '10Y-2Y Spread'),
    ('labor_market', 'UNRATE', 'Unemployment Rate'),
    ('labor_market', 'SAHM_Rule', 'Sahm Rule'),
    ('credit_spreads', 'HY_Spread', 'High Yield Spread'),
    ('gdp', 'GDP_YoY_Growth', 'GDP Growth (YoY)'),
    ('consumer', 'UMich_Sentiment', 'Consumer Sentiment'),
    ('housing', 'Housing_Starts', 'Housing Starts'),
]

for category, indicator, name in key_indicators:
    if category in latest and indicator in latest[category]:
        info = latest[category][indicator]
        value = info['value']
        date = info['date']
        
        # Format value
        if 'Growth' in indicator or 'Spread' in indicator or indicator == 'SAHM_Rule':
            value_str = f"{value:+.2f}%" if 'Growth' in indicator or 'Spread' in indicator else f"{value:.2f}"
        elif 'Rate' in name:
            value_str = f"{value:.1f}%"
        else:
            value_str = f"{value:.0f}"
        
        summary_data.append({
            'Indicator': name,
            'Current Value': value_str,
            'As of': date
        })

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*70)
print("LATEST INDICATOR VALUES")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)


FETCHING ALL RECESSION INDICATORS (CORE + SECONDARY)

FETCHING ALL CORE RECESSION INDICATORS

=== Fetching Treasury Yields ===
Loading DGS10 from cache...
Loading DGS2 from cache...
Loading DGS3MO from cache...

=== Fetching Labor Market Data ===
Loading UNRATE from cache...
Loading SAHMREALTIME from cache...
Loading ICSA from cache...

=== Fetching Credit Spreads ===
Loading BAMLH0A0HYM2 from cache...
Loading BAMLC0A4CBBB from cache...
Loading DBAA from cache...
Loading DAAA from cache...
Loading DGS10 from cache...

DATA FETCH COMPLETE

FETCHING ALL SECONDARY RECESSION INDICATORS

=== Fetching Leading Economic Index ===
Loading USSLIND from cache...

=== Fetching Manufacturing PMI ===
Loading MANEMP from cache...
Loading NEWORDER from cache...
Fetching NAPM from FRED API...
Error fetching NAPM: Bad Request.  The series does not exist.
NAPM not available
Loading INDPRO from cache...

=== Fetching GDP Data ===
Loading GDPC1 from cache...
Loading A939RX0Q048SBEA from cache...
Loading D

## 🔍 Cache Information

View data cache status and statistics.

In [17]:
# Display cache info
fetcher.print_cache_info()

print("\n💡 Tip: Data is cached based on update frequency (daily/weekly/monthly/quarterly)")
print("   To force refresh all data, run: fetcher.clear_cache()")

CACHE STATISTICS
Total cached series: 26
Valid caches: 26
Expired caches: 0
Total cache size: 0.27 MB

By frequency:
  daily: 7 series (refresh every 24h)
  monthly: 15 series (refresh every 720h)
  quarterly: 3 series (refresh every 2160h)
  weekly: 1 series (refresh every 168h)

Oldest cache: 2026-01-08T12:46:21.019837
Newest cache: 2026-01-08T12:48:40.628836

💡 Tip: Data is cached based on update frequency (daily/weekly/monthly/quarterly)
   To force refresh all data, run: fetcher.clear_cache()


---

## 🔄 Dashboard Refresh Instructions

To refresh the dashboard with the latest data:

1. **Quick Refresh**: Run "Cell" → "Run All" from the menu (or Ctrl+Shift+Enter)
2. **Force New Data**: Run `fetcher.clear_cache()` in the cache cell, then run all cells
3. **Selective Refresh**: Re-run individual cells to update specific visualizations

**Auto-refresh**: The data cache automatically expires based on indicator frequency:
- Daily indicators (yields): Refresh every 24 hours
- Weekly indicators (jobless claims): Refresh every 7 days  
- Monthly indicators (unemployment, PMI): Refresh every 30 days
- Quarterly indicators (GDP): Refresh every 90 days

---

**Dashboard Version**: 1.0  
**Data Source**: Federal Reserve Economic Data (FRED)  
**Recession Dates**: NBER Official Recession Periods